In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Conv2DTranspose, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/sampleSubmission.csv.zip


In [20]:
import pickle

In [21]:
# what is this?
%load_ext Cython

In [22]:
%%cython
cimport cython
import numpy as np

# substitute C equivalents for slow python objects
@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)

# cdefs can only be called by functions, not scripts.
cdef int calc_neighs(unsigned char[:, :] field, int i, int j, int n, int k):
    # i in (0, n)
    # j in (0, k)
    cdef:
        int neighs = 0;
        int i_min = i - 1;
        int i_pl = i + 1;
        int j_min = j - 1;
        int j_pl = j + 1;
    neighs = 0
    
    # top row
    if i_min >= 0:
        if j_min >= 0: 
            neighs += field[i_min, j_min]
        neighs += field[i_min, j]
        if j_pl < k:
            neighs += field[i_min, j_pl]
            
    # middle row
    if j_min >= 0:
        neighs += field[i, j_min]
    if j_pl < k:
        neighs += field[i, j_pl]
        
    # bottom row
    if i_pl < n:
        if j_min >= 0:
            neighs += field[i_pl, j_min]
        neighs += field[i_pl, j]
        if j_pl < k:
            neighs += field[i_pl, j_pl]
            
    # return neighbors for cell at (i, j)
    return neighs

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
# what is cpdef?
cpdef make_move(unsigned char[:, :] field, int moves):
    # current -> next : np.array
    cdef:
        int _, i, j, neighs;
        int n, k;
        int switch = 0; # this is used to swap roles of curr and next arrays.
                        # so you don't have to create a new copy of the current state each time
        unsigned char[:, :] cur_field;
        unsigned char[:, :] next_field;
    # copy input field
    cur_field = np.copy(field)
    # returns zeros with shape of fielod
    next_field = np.zeros_like(field, 'uint8') # uint8 is an 8-bit unsigned integer
    n = field.shape[0]
    k = field.shape[1]
    
    # moves = number of state transitions
    for _ in range(moves):
        if switch == 0:
            # for i,jth cell in field
            for i in range(n):
                for j in range(k):
                    # get number of live neighbors
                    neighs = calc_neighs(cur_field, i, j, n, k)
                    
                    # this is the rule implementation
                    if cur_field[i, j] and neighs == 2:
                        next_field[i, j] = 1
                    elif neighs == 3:
                        next_field[i, j] = 1
                    else:
                        next_field[i, j] = 0
        else:
            for i in range(n):
                for j in range(k):
                    neighs = calc_neighs(next_field, i, j, n, k)
                    if next_field[i, j] and neighs == 2:
                        cur_field[i, j] = 1
                    elif neighs == 3:
                        cur_field[i, j] = 1
                    else:
                        cur_field[i, j] = 0
                        
        # is this how he handles doing multiple transitions in one go? :/
        # oh, he just switches back and forth between curr/next.
        # on the second round, just use curr as next and next as curr
        # clever...?
        # why not just rename next to curr at the end?
        switch = (switch + 1) % 2
        
    # outputs a numpy array
    return np.array(next_field if switch else cur_field)

In [23]:
NROW, NCOL = 20, 20

def generate_samples(delta=1, n=32):
    """
    Generate batch of samples
    
    @return: (end_frames, start_frames)
    """
    # uint8 is an 8-bit unsigned integer
    # split into n batches of NROWxNCOL grids
    batch = np.split(np.random.binomial(1, 0.5, (NROW * n, NCOL)).astype('uint8'), n)
    Yy = [life.make_move(state, 5) for state in batch] # transition each state in batch
    # is this because we want to learn actual gol states, not just random starting states?
    Xx = [life.make_move(state, 1) for state in Yy] # transition each state in batch 
    Y = np.array([y.ravel() for y in Yy])
    X = np.array([x.ravel() for x in Xx])
    return X, Y
    
# this is the same function writen as a generator
def data_generator(delta=1, batch_size=32, ravel=True):
    """
    Can be used along with .fit_generator to generate training samples on the fly
    """
    while True:
        batch = np.split(np.random.binomial(1, 0.5, (NROW * batch_size, NCOL)).astype('uint8'), batch_size)
        Yy = [make_move(state, 5) for state in batch]
        Xx = [make_move(state, delta) for state in Yy]

        if ravel:
            # turn grids into 1D
            Y = np.array([y.ravel() for y in Yy])
            X = np.array([x.ravel() for x in Xx])
            yield X, Y
        else:
            yield np.array(Xx)[:,:, :, np.newaxis], np.array(Yy)[:, :, :, np.newaxis]

In [24]:
# try with different n_hidden_filters and kernel_size

def create_model(n_hidden_convs=2, n_hidden_filters=128, kernel_size=5):
    nn = Sequential()
    # hidden filters, kernel size, padding, activation?
    nn.add(Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu', input_shape=(20, 20, 1)))
    # what is batch normalization?
    nn.add(BatchNormalization())
    for i in range(n_hidden_convs):
        nn.add(Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu'))
        nn.add(BatchNormalization())
    nn.add(Conv2D(1, kernel_size, padding='same', activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn

def model_v2():
    # This one performs worse then pure Conv2D
    nn = Sequential()
    nn.add(Conv2D(128, 5, padding='same', activation=lrelu, input_shape=(20, 20, 1)))
    nn.add(BatchNormalization())
    nn.add(Conv2D(128, 5, padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(MaxPool2D())
    nn.add(Conv2DTranspose(128, 2, strides=(2, 2), padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(Conv2DTranspose(128, 5, strides=(1, 1), padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(Conv2D(1, 5, padding='same', activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn

In [25]:
models = []
for delta in range(1, 6):
    model = create_model(n_hidden_convs=6, n_hidden_filters=256)
    es = EarlyStopping(monitor='loss', patience=9, min_delta=0.001)
    model.fit_generator(data_generator(delta=delta, ravel=False), steps_per_epoch=500, epochs=50, verbose=1, callbacks=[es])
    models.append(model)
    
pickle.dump(models, open('models.pkl', 'wb'))

Epoch 1/50
500/500 [==============================] - 29s 57ms/step - loss: 0.3726 - acc: 0.8326
Epoch 2/50
500/500 [==============================] - 24s 47ms/step - loss: 0.2787 - acc: 0.8825
Epoch 3/50
500/500 [==============================] - 24s 49ms/step - loss: 0.2655 - acc: 0.8893
Epoch 4/50
500/500 [==============================] - 24s 47ms/step - loss: 0.2595 - acc: 0.8926
Epoch 5/50
500/500 [==============================] - 24s 48ms/step - loss: 0.2554 - acc: 0.8948
Epoch 6/50
500/500 [==============================] - 24s 48ms/step - loss: 0.2526 - acc: 0.8963
Epoch 7/50
500/500 [==============================] - 24s 47ms/step - loss: 0.2488 - acc: 0.8980
Epoch 8/50
500/500 [==============================] - 24s 48ms/step - loss: 0.2456 - acc: 0.8996
Epoch 9/50
500/500 [==============================] - 24s 47ms/step - loss: 0.2446 - acc: 0.9000
Epoch 10/50
500/500 [==============================] - 24s 47ms/step - loss: 0.2415 - acc: 0.9015
Epoch 11/50
500/500 [========

500/500 [==============================] - 24s 48ms/step - loss: 0.3779 - acc: 0.8315
Epoch 35/50
500/500 [==============================] - 24s 47ms/step - loss: 0.3780 - acc: 0.8315
Epoch 36/50
500/500 [==============================] - 24s 48ms/step - loss: 0.3793 - acc: 0.8309
Epoch 37/50
500/500 [==============================] - 24s 47ms/step - loss: 0.3771 - acc: 0.8320
Epoch 38/50
500/500 [==============================] - 24s 47ms/step - loss: 0.3761 - acc: 0.8327
Epoch 39/50
500/500 [==============================] - 24s 48ms/step - loss: 0.3765 - acc: 0.8321
Epoch 40/50
500/500 [==============================] - 24s 48ms/step - loss: 0.3760 - acc: 0.8324
Epoch 41/50
500/500 [==============================] - 24s 48ms/step - loss: 0.3763 - acc: 0.8325
Epoch 42/50
500/500 [==============================] - 24s 47ms/step - loss: 0.3761 - acc: 0.8325
Epoch 43/50
500/500 [==============================] - 24s 48ms/step - loss: 0.3755 - acc: 0.8328
Epoch 44/50
500/500 [===========

500/500 [==============================] - 24s 48ms/step - loss: 0.4710 - acc: 0.7854
Epoch 27/50
500/500 [==============================] - 24s 49ms/step - loss: 0.4707 - acc: 0.7855
Epoch 28/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4706 - acc: 0.7854
Epoch 29/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4708 - acc: 0.7855
Epoch 30/50
500/500 [==============================] - 24s 49ms/step - loss: 0.4706 - acc: 0.7856
Epoch 31/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4699 - acc: 0.7859
Epoch 1/50
500/500 [==============================] - 27s 55ms/step - loss: 0.5299 - acc: 0.7645
Epoch 2/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4976 - acc: 0.7774
Epoch 3/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4961 - acc: 0.7773
Epoch 4/50
500/500 [==============================] - 24s 48ms/step - loss: 0.4954 - acc: 0.7778
Epoch 5/50
500/500 [================

In [24]:
train_df = pd.read_csv('../input/train.csv.zip', index_col=0)
test_df = pd.read_csv('../input/test.csv.zip', index_col=0)

In [25]:
submit_df = pd.DataFrame(index=test_df.index, columns=['start.' + str(_) for _ in range(1, 401)])

In [ ]:
# test on one model
delta_df = test_df[test_df.delta==5].iloc[:, 1:].values.reshape(-1, 20, 20, 1)
submit_df[test_df.delta==5] = model.predict(delta_df).reshape(-1, 400).round(0).astype('uint8')

In [26]:
for delta in range(1, 6):
    mod = models[delta-1]
    delta_df = test_df[test_df.delta == delta].iloc[:, 1:].values.reshape(-1, 20, 20, 1)
    submit_df[test_df.delta == delta] = mod.predict(delta_df).reshape(-1, 400).round(0).astype('uint8')

In [2]:
submit_df

NameError: name 'submit_df' is not defined

In [ ]:
submit_df.to_csv('cnns_40.csv')

In [1]:
ls

__notebook_source__.ipynb


In [2]:
ls ..

input/  lib/  working/


In [ ]:
ls -a ../working